In [59]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

train = pd.read_csv('train_final.csv')
test = pd.read_csv('test_final.csv')
print("Trian and Test data shape",train.shape,test.shape)

(131662, 34) (87395, 34)


In [61]:
X = train.drop('Surge_Pricing_Type',axis=1)
y = train.Surge_Pricing_Type
X_final = test.drop('Surge_Pricing_Type',axis=1)
print(X.shape,y.shape,X_test.shape)

(131662, 33) (131662,) (17117, 33)


In [62]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier()

parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['multi:softmax'],
              'learning_rate': [0.001], #so called `eta` value
              'max_depth': [11],
              'min_child_weight': [7],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [1300], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [91]}

print("Model Ready")

clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(n_splits=10, shuffle=True), 
                   scoring='accuracy',
                   verbose=2, refit=True)

clf.fit(X, y)

print("Model Fitted")

Model Ready


In [65]:
preds = clf.predict(X_final)
df_submit = pd.read_csv('sample_submission.csv')
df_submit['Surge_Pricing_Type'] = preds
df_submit.to_csv('XGBBestTree.csv',index=False , header=True)
from sklearn.externals import joblib
joblib.dump(clf.best_estimator_, 'xgb.pkl')

In [34]:
## Getting best params from GridSearchCV
from sklearn.metrics import classification_report

print("Best parameters set found on development set:")
print(clf.best_params_)
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

print("Detailed classification report:")
y_true, y_pred = y, clf.predict(X)
print(classification_report(y_true, y_pred))

Best parameters set found on development set:


NameError: name 'clf' is not defined